# 3D Tomographic Reconstruction from 2D Flat Panel Detector Images

This notebook demonstrates the complete workflow for reconstructing 3D volumetric data from a series of 2D projections obtained from a flat panel detector.

## Overview
1. Data loading and preprocessing
2. Geometric setup and calibration
3. Reconstruction algorithm implementation
4. 3D visualization
5. Quality assessment
6. Export to 3D file formats

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import scipy.ndimage as ndimage
from skimage import io, transform, filters
import os
import sys
from tqdm import tqdm
import plotly.graph_objects as go

# Add the src directory to the path
sys.path.append('../src')

# Import our custom modules
from preprocessing import preprocess_projections
from reconstruction import filtered_backprojection, art_reconstruction, sirt_reconstruction, fdk_reconstruction
from visualization import show_projections, show_sinogram, show_volume_slices, show_volume_3slice, render_surface_plotly
from utils import load_tiff_stack, extract_angles_from_filenames, create_projection_geometry, save_numpy_as_vtk, save_volume_as_tiff_stack

## 1. Data Loading and Preprocessing

In this section, we load the 2D projections and perform preprocessing steps such as normalization, filtering, and artifact correction.

In [ ]:
# Define paths
data_path = '../data/raw/'
processed_path = '../data/processed/'
reconstructions_path = '../data/reconstructions/'

# Create directories if they don't exist
for path in [processed_path, reconstructions_path]:
    if not os.path.exists(path):
        os.makedirs(path)

# Parameters for loading data
file_pattern = '*.tif*'  # Pattern for TIFF files
angle_pattern = '_([\d.]+)deg'  # Pattern to extract angles from filenames

# Check if data exists
if os.path.exists(data_path) and len(os.listdir(data_path)) > 0:
    # Load all the 2D projection images
    projections, filenames = load_tiff_stack(data_path, file_pattern)
    
    # Extract angles from filenames
    angles = extract_angles_from_filenames(filenames, angle_pattern)
    
    print(f"Loaded {len(projections)} projections with shape {projections[0].shape}")
    print(f"Angle range: {angles.min():.2f}° to {angles.max():.2f}°")
    
    # Display sample projections
    indices = [0, len(projections)//3, 2*len(projections)//3, len(projections)-1]
    show_projections(projections, indices)
    
    # Display a sinogram
    show_sinogram(projections)
else:
    print("No data found in the raw data directory. Please add TIFF files before proceeding.")
    # For demonstration purposes, create a simple phantom
    print("Creating a simulation phantom for demonstration...")
    
    # Create a simple phantom (a cube with a sphere inside)
    phantom_size = 128
    phantom = np.zeros((phantom_size, phantom_size, phantom_size))
    
    # Add a cube
    margin = 20
    phantom[margin:-margin, margin:-margin, margin:-margin] = 0.5
    
    # Add a sphere
    x, y, z = np.ogrid[:phantom_size, :phantom_size, :phantom_size]
    center = phantom_size // 2
    radius = phantom_size // 4
    sphere = (x - center)**2 + (y - center)**2 + (z - center)**2 <= radius**2
    phantom[sphere] = 1.0
    
    # Create projection angles
    n_angles = 60
    angles = np.linspace(0, 360, n_angles, endpoint=False)
    
    # Create projections
    projections = np.zeros((n_angles, phantom_size, phantom_size))
    
    for i, angle in enumerate(angles):
        # Simple parallel beam projection (sum along rotated z-axis)
        rotated = ndimage.rotate(phantom, angle, axes=(0, 1), reshape=False, order=1)
        projections[i] = np.sum(rotated, axis=0)
    
    # Normalize projections to [0, 1]
    projections = (projections - projections.min()) / (projections.max() - projections.min())
    
    # Save the phantom and projections
    np.save(f"{reconstructions_path}/phantom.npy", phantom)
    np.save(f"{processed_path}/simulated_projections.npy", projections)
    
    # Display sample projections
    indices = [0, n_angles//3, 2*n_angles//3, n_angles-1]
    show_projections(projections, indices)
    
    # Display a sinogram
    show_sinogram(projections)

In [ ]:
# Preprocess the projections
processed_projections = preprocess_projections(
    projections,
    angles=angles,
    normalize=True,
    denoise=True,
    remove_rings=True,
    correct_rotation=True,
    log_transform=True
)

# Display preprocessed projections
show_projections(processed_projections, indices)

## 2. Geometric Setup and Calibration

Define the geometry of our imaging setup, including source-detector distance, rotation angles, etc.

In [ ]:
# Define geometric parameters
source_origin_dist = 500.0  # mm (distance from source to rotation center)
origin_detector_dist = 500.0  # mm (distance from rotation center to detector)
detector_shape = projections.shape[1:3]  # (height, width) of detector

# Setup projection geometry
geometry = create_projection_geometry(
    angles,
    detector_shape,
    source_origin_dist,
    origin_detector_dist
)

# Define the size of the reconstruction volume
volume_size = (detector_shape[0], detector_shape[0], detector_shape[0])  # cubic volume
print(f"Reconstruction volume size: {volume_size}")

## 3. Reconstruction Algorithm Implementation

Apply different reconstruction algorithms to create a 3D volume from 2D projections. We'll compare Filtered Back Projection (FBP), Algebraic Reconstruction Technique (ART), and Feldkamp-Davis-Kress (FDK) algorithm.

In [ ]:
# Filtered Back Projection (FBP) reconstruction
print("Starting Filtered Back Projection reconstruction...")
fbp_volume = filtered_backprojection(
    processed_projections,
    angles,
    volume_size
)
print("FBP reconstruction completed.")

# Save the reconstructed volume
np.save(f"{reconstructions_path}/fbp_volume.npy", fbp_volume)
save_numpy_as_vtk(fbp_volume, f"{reconstructions_path}/fbp_volume.vti")

# Display orthogonal slices of the reconstructed volume
show_volume_3slice(fbp_volume)

In [ ]:
# Algebraic Reconstruction Technique (ART)
print("Starting ART reconstruction...")
art_volume = art_reconstruction(
    processed_projections,
    angles,
    volume_size,
    iterations=5  # ART is iterative, so we specify the number of iterations
)
print("ART reconstruction completed.")

# Save the reconstructed volume
np.save(f"{reconstructions_path}/art_volume.npy", art_volume)
save_numpy_as_vtk(art_volume, f"{reconstructions_path}/art_volume.vti")

# Display orthogonal slices of the reconstructed volume
show_volume_3slice(art_volume)

In [ ]:
# SIRT (Simultaneous Iterative Reconstruction Technique)
print("Starting SIRT reconstruction...")
sirt_volume = sirt_reconstruction(
    processed_projections,
    angles,
    volume_size,
    iterations=5
)
print("SIRT reconstruction completed.")

# Save the reconstructed volume
np.save(f"{reconstructions_path}/sirt_volume.npy", sirt_volume)
save_numpy_as_vtk(sirt_volume, f"{reconstructions_path}/sirt_volume.vti")

# Display orthogonal slices of the reconstructed volume
show_volume_3slice(sirt_volume)

In [ ]:
# For cone-beam CT, use FDK algorithm
print("Starting FDK reconstruction...")
fdk_volume = fdk_reconstruction(
    processed_projections,
    geometry,
    volume_size
)
print("FDK reconstruction completed.")

# Save the reconstructed volume
np.save(f"{reconstructions_path}/fdk_volume.npy", fdk_volume)
save_numpy_as_vtk(fdk_volume, f"{reconstructions_path}/fdk_volume.vti")

# Display orthogonal slices of the reconstructed volume
show_volume_3slice(fdk_volume)

## 4. 3D Visualization

In this section, we visualize the reconstructed 3D volume.

In [ ]:
# Create interactive 3D visualization with Plotly
# Adjust threshold as needed (higher threshold for sparser result)
try:
    threshold = 0.5 * fbp_volume.max()
    fig = render_surface_plotly(fbp_volume, threshold=threshold, opacity=0.7)
    fig.write_html(f"{reconstructions_path}/fbp_isosurface.html")
    fig.show()
except Exception as e:
    print(f"Error creating 3D visualization: {e}")

In [ ]:
# Export as TIFF stack for compatibility with other 3D software
save_volume_as_tiff_stack(fbp_volume, f"{reconstructions_path}/fbp_slices", "slice")
print(f"Saved TIFF stack to {reconstructions_path}/fbp_slices/")

## 5. Comparison and Quality Assessment

Compare the results from different reconstruction algorithms and assess their quality.

In [ ]:
# Compare middle slices of different reconstruction methods
methods = ['FBP', 'ART', 'SIRT', 'FDK']
volumes = [fbp_volume, art_volume, sirt_volume, fdk_volume]

# Get middle slice
mid_z = volume_size[2] // 2

plt.figure(figsize=(15, 4))
for i, (method, vol) in enumerate(zip(methods, volumes)):
    plt.subplot(1, 4, i+1)
    plt.imshow(vol[:, :, mid_z], cmap='gray')
    plt.title(f"{method} Reconstruction")
    plt.colorbar()
plt.tight_layout()
plt.savefig(f"{reconstructions_path}/method_comparison.png", dpi=300)
plt.show()

In [ ]:
# Calculate metrics between reconstruction methods
def calculate_metrics(vol1, vol2):
    """Calculate MSE and PSNR between two volumes."""
    mse = np.mean((vol1 - vol2) ** 2)
    if mse == 0:
        return mse, float('inf')
    max_val = max(vol1.max(), vol2.max())
    psnr = 20 * np.log10(max_val / np.sqrt(mse))
    return mse, psnr

print("Comparing reconstruction methods:")
for i in range(len(methods)):
    for j in range(i+1, len(methods)):
        method1, vol1 = methods[i], volumes[i]
        method2, vol2 = methods[j], volumes[j]
        
        mse, psnr = calculate_metrics(vol1, vol2)
        
        print(f"{method1} vs {method2}:")
        print(f"  MSE: {mse:.6f}")
        print(f"  PSNR: {psnr:.2f} dB")

## 6. Performance Analysis

Analyze the performance of the different reconstruction algorithms in terms of computation time and memory usage.

In [ ]:
import time
import psutil
import gc

def measure_performance(func, *args, **kwargs):
    """Measure execution time and memory usage of a function."""
    # Force garbage collection before measurement
    gc.collect()
    
    # Get initial memory usage
    process = psutil.Process()
    mem_before = process.memory_info().rss / 1024 / 1024  # MB
    
    # Measure execution time
    start_time = time.time()
    result = func(*args, **kwargs)
    end_time = time.time()
    
    # Get final memory usage
    mem_after = process.memory_info().rss / 1024 / 1024  # MB
    
    # Calculate metrics
    elapsed_time = end_time - start_time
    mem_increase = mem_after - mem_before
    
    return result, elapsed_time, mem_increase

# Only run performance tests if we have real data (not simulated data)
if 'phantom' not in locals():
    # Define a smaller volume size for testing
    test_size = (64, 64, 64)
    test_projections = processed_projections[:, ::2, ::2]  # Downsample for faster testing
    
    # Test all reconstruction methods
    methods = [
        ('FBP', lambda: filtered_backprojection(test_projections, angles, test_size)),
        ('ART', lambda: art_reconstruction(test_projections, angles, test_size, iterations=3)),
        ('SIRT', lambda: sirt_reconstruction(test_projections, angles, test_size, iterations=3)),
        ('FDK', lambda: fdk_reconstruction(test_projections, geometry, test_size))
    ]
    
    results = []
    for name, func in methods:
        try:
            print(f"Testing {name}...")
            _, elapsed_time, mem_increase = measure_performance(func)
            results.append((name, elapsed_time, mem_increase))
            print(f"  Time: {elapsed_time:.2f} seconds")
            print(f"  Memory increase: {mem_increase:.2f} MB")
        except Exception as e:
            print(f"Error testing {name}: {e}")
    
    # Plot performance results
    plt.figure(figsize=(12, 5))
    
    # Time plot
    plt.subplot(1, 2, 1)
    names = [r[0] for r in results]
    times = [r[1] for r in results]
    plt.bar(names, times)
    plt.title('Execution Time')
    plt.ylabel('Time (seconds)')
    
    # Memory plot
    plt.subplot(1, 2, 2)
    memories = [r[2] for r in results]
    plt.bar(names, memories)
    plt.title('Memory Usage')
    plt.ylabel('Memory Increase (MB)')
    
    plt.tight_layout()
    plt.savefig(f"{reconstructions_path}/performance_comparison.png", dpi=300)
    plt.show()

## 7. Conclusion

In this notebook, we have demonstrated the process of 3D tomographic reconstruction from 2D projections using various algorithms. The key findings are:

1. FBP is generally the fastest method but may produce more artifacts in noisy data.
2. Iterative methods (ART, SIRT) can produce better results with noisy data but require more computational resources.
3. FDK is well-suited for cone-beam CT geometry.

The output of our reconstruction pipeline includes:
- VTK files (.vti) that can be opened with 3D visualization software like ParaView or VTK.js
- TIFF stacks that can be imported into various 3D software packages
- NumPy arrays (.npy) for further processing in Python

The results show that [insert your findings based on the specific reconstruction results].

Future improvements could include:
- Implementation of more advanced iterative algorithms
- GPU acceleration for faster reconstruction
- Deep learning-based reconstruction approaches
- More sophisticated artifact correction methods